In [1]:
from datetime import datetime
import time
import pickle
import glob

import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from skimage.io import imread
from skimage.transform import resize
from tensorflow.python.framework.ops import reset_default_graph

from util import etl, helpers

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
sess = tf.InteractiveSession()
# loading data and setting up constants
TRAIN_PATH = "./data/train.csv"
TEST_PATH = "./data/test.csv"
IMAGE_PATHS = glob.glob("./data/images/*.jpg")
IMAGE_SHAPE = (128, 128, 1)

with open('./data/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('./data/test.pickle', 'rb') as f:
    test = pickle.load(f)
with open('./data/le.pickle', 'rb') as f:
    le = pickle.load(f)

In [38]:
#Training Loop
BATCH_SIZE = 64
ITERATIONS = 1e4
LOG_FREQ = 10
VALIDATION_SIZE = 0.1 # 0.1 is ~ 100 samples for valition
SEED = 42
DROPOUT = False
LEARNING_RATE = 0.0005
VALID_EVERY = 100
TRAIN_SIZE = 0.2

NUM_CLASSES = 99   # len(train['species'].unique())
NUM_FEATURES = 64 # for all three features, margin, shape and texture


In [37]:
height, width, channels = IMAGE_SHAPE
reset_default_graph()

x_image_pl = tf.placeholder(tf.float32, [None, height, width, channels], name="x_image_pl")
x_flat_image_pl = tf.placeholder(tf.float32, [None, height * width * channels], name='x_flat_image_pl')
x_margin_pl = tf.placeholder(tf.float32, [None, NUM_FEATURES], name="x_margin_pl")
x_shape_pl = tf.placeholder(tf.float32, [None, NUM_FEATURES], name="x_shape_pl")
x_texture_pl = tf.placeholder(tf.float32, [None, NUM_FEATURES], name="x_texture_pl")

ts_pl = tf.placeholder(tf.float32, [None, NUM_CLASSES], name="targets_pl")

x = tf.concat(concat_dim=1, values=[x_margin_pl, x_shape_pl, x_texture_pl], name='features')

W = tf.Variable(tf.zeros([(3*NUM_FEATURES), NUM_CLASSES]))
b = tf.Variable(tf.zeros([NUM_CLASSES]))

sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


y = tf.matmul(x,W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, ts_pl))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [39]:
batch_gen = etl.batch_generator(train, test, batch_size=BATCH_SIZE, num_classes=NUM_CLASSES,
                            num_iterations=ITERATIONS, seed=SEED, train_size=TRAIN_SIZE, val_size=VALIDATION_SIZE)

train_batch = batch_gen.gen_train()
valid_batch = batch_gen.gen_valid()
test_batch = batch_gen.gen_test()

prev = time.time()
for i, batch in enumerate(batch_gen.gen_train()):
    train_step.run(
        feed_dict={
            x: batch['features'],
            ts_pl: batch['ts']
        })
    #print("step %d, training accuracy %g"%(i, train_accuracy))
    if i % 1000 == 0:
        print("step %d %.2f seconds" % (i, time.time() - prev))
        prev = time.time()
    if i >= ITERATIONS:
        break
save_path = saver.save(sess, "%s_model.ckpt" % datetime.now().isoformat())
print 'done'

initiating batch generator
batch generator initiated ...
step 0 0.13 seconds
step 1000 7.24 seconds
step 2000 6.59 seconds
step 3000 8.32 seconds
step 4000 7.78 seconds
step 5000 6.83 seconds
step 6000 6.49 seconds
step 7000 6.55 seconds
step 8000 6.39 seconds
step 9000 6.46 seconds
step 10000 6.71 seconds
done


In [33]:
#correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
loss, accuracy, argmax_y = helpers.loss_and_acc(ts_pl, y)
valid_batch, i = valid_batch.next()
print(accuracy.eval(feed_dict={x: valid_batch['features'], ts_pl: valid_batch['ts']}))

0.015625


In [36]:
reload(helpers)
helpers.predictions(sess, batch_gen, y, [x_margin_pl, x_shape_pl, x_texture_pl], ['margins', 'shapes', 'textures'], le.classes_)

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,Acer_Saccharinum,Alnus_Cordata,Alnus_Maximowiczii,Alnus_Rubra,Alnus_Sieboldiana,Alnus_Viridis,Arundinaria_Simonii,Betula_Austrosinensis,Betula_Pendula,Callicarpa_Bodinieri,
0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
5,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
6,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
7,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
8,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
9,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
